<a href="https://colab.research.google.com/github/satyajitghana/TSAI-DeepVision-EVA4.0-Phase-2/blob/master/13-AI4Sound/Speech_Recognition_Inferencing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install torchaudio==0.7.0

     |████████████████████████████████| 7.6MB 5.8MB/s 


In [2]:
! pip install SoundFile

In [3]:
import gdown

url = 'https://drive.google.com/uc?id=12MmbJioSAA-hs5y-RpHF0-Al5nAHdcgG'
output = 'speech-recognition-residual-model.scripted.pt'
gdown.download(url, output, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=12MmbJioSAA-hs5y-RpHF0-Al5nAHdcgG
To: /content/speech-recognition-residual-model.scripted.pt
94.9MB [00:00, 129MB/s] 


'speech-recognition-residual-model.scripted.pt'

In [4]:
! wget -O speech_commands_v0.01.tar.gz http://download.tensorflow.org/data/speech_commands_v0.01.tar.gz
! tar xzf speech_commands_v0.01.tar.gz 
! ls

--2020-11-25 19:10:24--  http://download.tensorflow.org/data/speech_commands_v0.01.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 173.194.217.128, 2607:f8b0:400c:c0d::80
Connecting to download.tensorflow.org (download.tensorflow.org)|173.194.217.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1489096277 (1.4G) [application/gzip]
Saving to: ‘speech_commands_v0.01.tar.gz’

speech_commands_v0. 100%[===================>]   1.39G   242MB/s    in 7.1s    

2020-11-25 19:10:32 (200 MB/s) - ‘speech_commands_v0.01.tar.gz’ saved [1489096277/1489096277]

_background_noise_  LICENSE	 speech_commands_v0.01.tar.gz
bed		    marvin	 speech-recognition-residual-model.scripted.pt
bird		    nine	 stop
cat		    no		 testing_list.txt
dog		    off		 three
down		    on		 tree
eight		    one		 two
five		    README.md	 up
four		    right	 validation_list.txt
go		    sample_data  wow
happy		    seven	 yes
house		    sheila	 zero
left		    six


In [5]:
import torch
import torchaudio
import soundfile as sf
import io

/usr/local/lib/python3.6/dist-packages/torchaudio/backend/utils.py:54: UserWarning: "sox" backend is being deprecated. The default backend will be changed to "sox_io" backend in 0.8.0 and "sox" backend will be removed in 0.9.0. Please migrate to "sox_io" backend. Please refer to https://github.com/pytorch/audio/issues/903 for the detail.
  '"sox" backend is being deprecated. '


In [6]:
class TextTransform:
    """Maps characters to integers and vice versa"""
    def __init__(self):
        char_map_str = """
        ' 0
        <SPACE> 1
        a 2
        b 3
        c 4
        d 5
        e 6
        f 7
        g 8
        h 9
        i 10
        j 11
        k 12
        l 13
        m 14
        n 15
        o 16
        p 17
        q 18
        r 19
        s 20
        t 21
        u 22
        v 23
        w 24
        x 25
        y 26
        z 27
        """
        self.char_map = {}
        self.index_map = {}
        for line in char_map_str.strip().split('\n'):
            ch, index = line.split()
            self.char_map[ch] = int(index)
            self.index_map[int(index)] = ch
        self.index_map[1] = ' '

    def text_to_int(self, text):
        """ Use a character map and convert text to an integer sequence """
        int_sequence = []
        for c in text:
            if c == ' ':
                ch = self.char_map['<SPACE>']
            else:
                ch = self.char_map[c]
            int_sequence.append(ch)
        return int_sequence

    def int_to_text(self, labels):
        """ Use a character map and convert integer labels to an text sequence """
        string = []
        for i in labels:
            string.append(self.index_map[i])
        return ''.join(string).replace('<SPACE>', ' ')

In [7]:
text_transform = TextTransform()

In [8]:
model = torch.jit.load("/content/speech-recognition-residual-model.scripted.pt", map_location="cpu")

In [9]:
audio_transform = torchaudio.transforms.MelSpectrogram() 

/usr/local/lib/python3.6/dist-packages/torchaudio/functional.py:318: UserWarning: At least one mel filterbank has all zero values. The value for `n_mels` (128) may be set too high. Or, the value for `n_freqs` (201) may be set too low.
  "At least one mel filterbank has all zero values. "


In [161]:
# waveform, sample_rate = torchaudio.load("cat/004ae714_nohash_0.wav", normalization=True)
waveform, _ = torchaudio.load("/content/stop/004ae714_nohash_0.wav", normalization=True) # its normalized by default anyways
input_audio = audio_transform(waveform)
with torch.no_grad():
    out = model(input_audio.unsqueeze(0))
arg_maxes = torch.argmax(out, dim=2)
decodes = []
targets = []
blank_label = 28
collapse_repeated = True
for i, args in enumerate(arg_maxes):
    decode = []
    for j, index in enumerate(args):
        if index != blank_label:
            if collapse_repeated and j != 0 and index == args[j -1]:
                continue
            decode.append(index.item())
    decodes.append(text_transform.int_to_text(decode))
decodes[0]

'stop '

In [35]:
in_file = open("/content/happy/07363607_nohash_0.wav", "rb") # opening for [r]eading as [b]inary
data = in_file.read()

In [116]:
data, sample_rate = sf.read(io.BytesIO(data))
waveform, _ = torch.from_numpy(data).float(), sample_rate
waveform = waveform.unsqueeze(0)

In [117]:
type(waveform)

torch.Tensor

In [125]:
input_audio = audio_transform(waveform)

In [126]:
input_audio.shape

torch.Size([1, 128, 81])

In [127]:
with torch.no_grad():
    out = model(input_audio.unsqueeze(0))

In [128]:
arg_maxes = torch.argmax(out, dim=2)

In [129]:
arg_maxes

tensor([[28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28,  5, 28, 16, 28, 28, 28,
         28, 28, 28, 28, 28, 28, 28, 28,  8, 28, 28, 28, 28, 28, 28, 28, 28, 28,
         28, 28, 28, 28, 28]])

In [130]:
decodes = []
targets = []
blank_label = 28
collapse_repeated = True
for i, args in enumerate(arg_maxes):
    decode = []
    for j, index in enumerate(args):
        if index != blank_label:
            if collapse_repeated and j != 0 and index == args[j -1]:
                continue
            decode.append(index.item())
    decodes.append(text_transform.int_to_text(decode))

In [131]:
decodes[0]

'dog'